## Testing Temporal Events

To Run the os required to run code

Open linux/ubuntu terminal and run `jupyter notebook`

If this command doesnt exist, install the requirements: `pip install -U jupyter`

In [1]:
# ! pip install update

ERROR: Could not find a version that satisfies the requirement libaio-dev (from versions: none)
ERROR: No matching distribution found for libaio-dev


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("API_KEY")

Extract text to be read for triple extraction

In [10]:
from openai import OpenAI

# models:
#   - gpt-3.5-turbo
#   - gpt-4
#   - gpt-4-turbo
#   - gpt-4o-mini
def query(prompt, model='gpt-4o-mini', temperature=0):
    client = OpenAI(api_key=api_key)

    res = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=[
            {"role": "user", "content": prompt}
            ])

    # print(prompt)
    print('output is: ', res)
    print()
    print()

    return res


In [58]:
text = """ 
I want to generate training data for a Named Entity Recognition (NER) model designed to extract geological time periods and events from unstructured text. Please generate 1000 unique input-output pairs where the input contains geological time periods and events. The output should include tokenized sentences and corresponding labels in the BIO format.

Requirements:

Sentence Structure: Vary the position of the geological time periods and events within the sentence. Avoid starting every sentence with the word "The."
Geological Time Formats: Include variations such as "1070Ma," "1070 Ma," "1030-950 Ma," "(1030-950 Ma)," "~570Ma," "1070 to 755Ma," "2.2 Ga," "Triassic," etc.
Unit Variability: Include different units like Ma (Megaannum), Ga (Gigaannum), ka (Kiloannum), as well as geological timescales like "Triassic."
Context: Ensure the geological time periods are embedded in a relevant geological event or description. Use context such as orogenies, basin formation, or tectonic events.
Output Consistency: The number of tokens in the output labels must match the number of tokens in the input sentence. Use "O" for non-entity tokens, "B-GEO_TIME" for the beginning of a geological time entity, "I-GEO_TIME" for the continuation of the same entity, and "B-TIMESCALE" for geological timescales like "Triassic."
Entity Labeling: Ensure that only the geological time-related tokens (e.g., numbers followed by units like "Ma," "Ga") are labeled. Non-geological time-related entities (e.g., "Andes") should not be labeled.

Examples:

Input: "The rocks were formed around 1070 Ma during the Capricorn Orogeny."
Output: ["The", "rocks", "were", "formed", "around", "1070", "Ma", "during", "the", "Capricorn", "Orogeny", "."]
Labels: ["O", "O", "O", "O", "O", "B-GEO_TIME", "I-GEO_TIME", "O", "O", "O", "O", "O"]

Input: "Deformation occurred between 1830 and 1780 Ma as part of the Mangaroon Orogeny."
Output: ["Deformation", "occurred", "between", "1830", "and", "1780", "Ma", "as", "part", "of", "the", "Mangaroon", "Orogeny", "."]
Labels: ["O", "O", "O", "B-GEO_TIME", "O", "B-GEO_TIME", "I-GEO_TIME", "O", "O", "O", "O", "O", "O", "O"]

Input:"The Yilgarn Craton collision happened during the Triassic period around 250 Ma."
Output: ["The", "Yilgarn", "Craton", "collision", "happened", "during", "the", "Triassic", "period", "around", "250", "Ma", "."]
Labels: ["O", "O", "O", "O", "O", "O", "O", "B-TIMESCALE", "O", "O", "B-GEO_TIME", "I-GEO_TIME", "O"]

Focus more on GEO_TIME entities and less on TIMESCALE entities.

Generate 100 such unique examples, ensuring a mix of different geological time formats, sentence structures, and contexts.
"""

In [59]:
time_ml_annotation = query(text).choices[0].message.content.strip()

output is:  ChatCompletion(id='chatcmpl-9xV6xNc2D4tLOeaFAsCPCpbfW5tCf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here are 100 unique input-output pairs for training a Named Entity Recognition (NER) model focused on geological time periods and events:\n\n1. **Input:** "Around 1070 Ma, the region experienced significant volcanic activity."\n   - **Output:** ["Around", "1070", "Ma", ",", "the", "region", "experienced", "significant", "volcanic", "activity", "."]\n   - **Labels:** ["0", "B-GEO_TIME", "I-GEO_TIME", "0", "0", "0", "0", "0", "0", "0", "0"]\n\n2. **Input:** "The formation of the supercontinent Pannotia occurred approximately 600 Ma."\n   - **Output:** ["The", "formation", "of", "the", "supercontinent", "Pannotia", "occurred", "approximately", "600", "Ma", "."]\n   - **Labels:** ["0", "0", "0", "0", "0", "0", "0", "0", "B-GEO_TIME", "I-GEO_TIME", "0"]\n\n3. **Input:** "During the Triassic period, around 250 Ma, dinosau

In [60]:
print(time_ml_annotation)

Here are 100 unique input-output pairs for training a Named Entity Recognition (NER) model focused on geological time periods and events:

1. **Input:** "Around 1070 Ma, the region experienced significant volcanic activity."
   - **Output:** ["Around", "1070", "Ma", ",", "the", "region", "experienced", "significant", "volcanic", "activity", "."]
   - **Labels:** ["0", "B-GEO_TIME", "I-GEO_TIME", "0", "0", "0", "0", "0", "0", "0", "0"]

2. **Input:** "The formation of the supercontinent Pannotia occurred approximately 600 Ma."
   - **Output:** ["The", "formation", "of", "the", "supercontinent", "Pannotia", "occurred", "approximately", "600", "Ma", "."]
   - **Labels:** ["0", "0", "0", "0", "0", "0", "0", "0", "B-GEO_TIME", "I-GEO_TIME", "0"]

3. **Input:** "During the Triassic period, around 250 Ma, dinosaurs began to dominate."
   - **Output:** ["During", "the", "Triassic", "period", ",", "around", "250", "Ma", ",", "dinosaurs", "began", "to", "dominate", "."]
   - **Labels:** ["0", "0

In [57]:
import json

# Initialize lists to store outputs and labels
outputs = []
labels = []

lines = time_ml_annotation.splitlines()

# Iterate over each line and extract Output and Labels
for line in lines:
    if "**Output:**" in line:
        output = eval(line.split("**Output:** ")[1].strip())
        outputs.append(output)
    elif "**Labels:**" in line:
        label = eval(line.split("**Labels:** ")[1].strip())
        labels.append(label)

# Combine the outputs and labels into a list of tuples
output_label_pairs = list(zip(outputs, labels))

# Print the results
print("Outputs:", outputs)
print("Labels:", labels)
print("Output-Label Pairs:", output_label_pairs)

Outputs: [['Around', '1070', 'Ma', ',', 'the', 'region', 'experienced', 'significant', 'volcanic', 'activity', '.'], ['The', 'formation', 'of', 'the', 'supercontinent', 'Pannotia', 'occurred', 'approximately', '600', 'Ma', '.'], ['During', 'the', 'Triassic', ',', 'around', '250', 'Ma', ',', 'dinosaurs', 'began', 'to', 'dominate', '.'], ['The', 'last', 'glacial', 'maximum', 'occurred', 'roughly', '20', 'ka', 'ago', '.'], ['Evidence', 'suggests', 'that', 'the', 'Andean', 'orogeny', 'began', 'around', '200', 'Ma', '.'], ['The', 'Cambrian', 'explosion', 'took', 'place', 'approximately', '541', 'Ma', '.'], ['Around', '2.2', 'Ga', ',', 'the', "Earth's", 'crust', 'began', 'to', 'stabilize', '.'], ['The', 'formation', 'of', 'the', 'Himalayas', 'is', 'linked', 'to', 'events', 'around', '50', 'Ma', '.'], ['The', 'Permian', 'period', 'ended', 'about', '252', 'Ma', 'with', 'a', 'mass', 'extinction', '.'], ['The', 'last', 'major', 'tectonic', 'shift', 'occurred', 'around', '65', 'Ma', '.'], ['The',

In [61]:
data = [{"output": output, "labels": labels} for output, labels in output_label_pairs]
print(data)

# Specify the file name
file_name = "NER/Training_Data/geotimeLabels.json"

# Write the data to a JSON file
with open(file_name, 'w') as json_file:
    json.dump(data, json_file)

[{'output': ['Around', '1070', 'Ma', ',', 'the', 'region', 'experienced', 'significant', 'volcanic', 'activity', '.'], 'labels': ['0', 'B-GEO_TIME', 'I-GEO_TIME', '0', '0', '0', '0', '0', '0', '0', '0']}, {'output': ['The', 'formation', 'of', 'the', 'supercontinent', 'Pannotia', 'occurred', 'approximately', '600', 'Ma', '.'], 'labels': ['0', '0', '0', '0', '0', '0', '0', '0', 'B-GEO_TIME', 'I-GEO_TIME', '0']}, {'output': ['During', 'the', 'Triassic', ',', 'around', '250', 'Ma', ',', 'dinosaurs', 'began', 'to', 'dominate', '.'], 'labels': ['0', '0', 'B-TIMESCALE', '0', '0', 'B-GEO_TIME', 'I-GEO_TIME', '0', '0', '0', '0', '0']}, {'output': ['The', 'last', 'glacial', 'maximum', 'occurred', 'roughly', '20', 'ka', 'ago', '.'], 'labels': ['0', '0', '0', '0', '0', '0', 'B-GEO_TIME', 'I-GEO_TIME', '0', '0']}, {'output': ['Evidence', 'suggests', 'that', 'the', 'Andean', 'orogeny', 'began', 'around', '200', 'Ma', '.'], 'labels': ['0', '0', '0', '0', '0', '0', '0', '0', 'B-GEO_TIME', 'I-GEO_TIME'

In [12]:
# Define the TimeML template
time_ml_template = (
    "The following text contains various events and temporal expressions. "
    "Your task is to annotate these events and temporal expressions with TimeML tags. "
    "Follow the TimeML annotation guidelines. \n"
    "---------------------\n"
    f"Text: \n{text}\n"
    "TimeML Annotation:\n"
)

# Query for TimeML annotations
time_ml_annotation = query(time_ml_template).choices[0].message.content.strip()

output is:  ChatCompletion(id='chatcmpl-9qdwx7pg5obULxl2lRVinfPxtSD6V', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="<EVENT>Elon Musk</EVENT> is a billionaire entrepreneur and CEO of <EVENT>SpaceX</EVENT> and <EVENT>Tesla</EVENT>. He is known for his ambitious projects and innovative ideas. Musk has been involved in various industries, including electric vehicles, space exploration, and renewable energy. He is also a proponent of artificial intelligence and has expressed concerns about its potential risks. Musk's companies have made significant advancements in their respective fields, and he continues to push the boundaries of technology and innovation.", role='assistant', function_call=None, tool_calls=None))], created=1722333651, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=105, prompt_tokens=135, total_tokens=240))




In [8]:
prompt = """
Generate 500 sentences related to geological topics with placeholders for various entities. The entities should be: {rocks}, {minerals}, {stratigraphy}, {ores_deposits}, {locations}, and {geological_timescales}. Create a diverse set of sentences that can be used to describe geological features, formations, and discoveries. Vary the number of entities in each sentence, sometimes using more than three entities. Do not strictly follow the same patterns as the examples below; feel free to create your own patterns inspired by these ideas. Here are a few examples:

1. "The {rocks} in this region are primarily composed of {minerals}."
2. "During the {geological_timescales}, significant {ores_deposits} were formed."
3. "Stratigraphic analysis shows that these {stratigraphy} layers contain abundant {minerals}."
4. "These {rocks} have been found in {locations}."
5. "The {locations} are known for their rich deposits of {ores_deposits}."
6. "Geologists discovered {minerals} in the {stratigraphy} layers of this region."
7. "In the {geological_timescales}, {rocks} and {ores_deposits} were frequently found in {locations}."
8. "The {stratigraphy} of the {locations} reveals the presence of {minerals} and {rocks} from various periods."
9. "During the {geological_timescales}, both {rocks} and {rocks} were discovered in {locations}, indicating rich geological activity."

Please generate the remaining sentences with a mix of your own patterns and structures, ensuring a variety of sentence styles.
"""

# Query for geological sentences
# geological_sentences = query(prompt).choices[0].message.content.strip()

output is:  ChatCompletion(id='chatcmpl-9qyTXcmdL8BWENRVYGEl6T2DvDy4f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Sure! Here are 500 sentences related to geological topics with placeholders for various entities:\n\n1. "The {rocks} found in {locations} are primarily igneous in origin."\n2. "Analysis of the {stratigraphy} in {locations} reveals a complex history of {minerals} deposition."\n3. "Significant {ores_deposits} were identified during the {geological_timescales} in {locations}."\n4. "The presence of {minerals} in these {rocks} suggests a volcanic origin."\n5. "In {locations}, the {stratigraphy} indicates a transition from marine to terrestrial environments."\n6. "The {geological_timescales} show that {rocks} were formed under high-pressure conditions."\n7. "Explorations in {locations} have uncovered new {ores_deposits} of {minerals}."\n8. "The {rocks} in this area are rich in {minerals} that are essential for industrial 

In [24]:
import re 
from pathlib import Path

def write_to_file(geological_sentences, filename):

    sentences = geological_sentences.strip().split('\n')
    with open(filename, 'w') as file:
        for line in sentences:
            if '"' in line:
                line = re.findall(r'"(.*?)"', line)[0]
                file.write(line + '\n')

# filename = Path("NER") / "DataExamples.txt"
# write_to_file(geological_sentences, filename)

In [7]:
# Define the TimeML template
time_ml_template = (
    "The following text contains various events and temporal expressions. "
    "Your task is to annotate these events and temporal expressions with TimeML tags. "
    "Follow the TimeML annotation guidelines. \n"
    "---------------------\n"
    "Example:\n"
    "H. Marshall Schwarz was <EVENT eid='e1' class='OCCURRENCE'>named</EVENT> chairman and chief executive officer of U.S. Trust Corp., a private-banking firm with assets under management of about $17 billion.\n"
    "---------------------\n"
    f"Text: \n{text}\n"
    "TimeML Annotation:\n"
    "Give me the list of words such as 'named' that are temporal expressions or events."
)

# Query for TimeML annotations
test_a = query(time_ml_template).choices[0].message.content.strip()

output is:  ChatCompletion(id='chatcmpl-9EZFyuzMeJuTklhOMopzugQEr8frU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. born\n2. immigrating\n3. matriculated\n4. transferred\n5. received\n6. moved\n7. dropped out\n8. co-founded\n9. acquired\n10. proposed\n11. co-founded\n12. founded\n13. acquired\n14. merged\n15. acquired\n16. founded\n17. expressed\n18. criticized\n19. sued\n20. settled\n21. stepped down\n22. born\n23. divorced\n24. regretted\n25. became\n26. beaten\n27. hospitalized\n28. developed\n29. sold\n30. attended\n31. graduated\n32. applied\n33. immigrate\n34. connected\n35. worked\n36. entered\n37. transferred\n38. earned\n39. awarded\n40. hosted\n41. wrote\n42. held\n43. accepted\n44. decided\n45. founded\n46. became\n47. attempted\n48. failed\n49. succeeded\n50. received\n51. launched\n52. docked\n53. landed\n54. achieved\n55. launched\n56. began\n57. provided\n58. stated\n59. sent\n60. stated\n61. met\n62. sued\n63. 

In [10]:
# Define the TimeML template including provided annotations
time_ml_template = (
    "The following text contains various events and temporal expressions. "
    "Your task is to annotate these events and temporal expressions with TimeML tags. "
    "Follow the TimeML annotation guidelines provided below. \n"
    "---------------------\n"
    "<?xml version=\"1.0\" ?>\n"
    "<TimeML xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"http://timeml.org/timeMLdocs/TimeML_1.2.1.xsd\">\n"
    "<EXTRAINFO>\n"
    "Provide any additional information here\n"
    "</EXTRAINFO>\n"
    "<TEXTO>\n"
    "annotated text goes here - an example: 'H. Marshall Schwarz was <EVENT eid='e1' class='OCCURRENCE'>named</EVENT> chairman'\n"
    "</TEXTO>\n"
    "{Your provided TimeML annotations go here}\n"
    "</TimeML>"
    "Annotate the following text using the TimeML annotations by detecting the events and temporal expressions. \n"
    "{Your text goes here}\n"
)

# Define the provided TimeML annotations
provided_time_ml_annotations = """\
These are some examples of TimeML annotations that you might have received from the annotators.
<MAKEINSTANCE eventID="e1" eiid="ei2408" tense="PAST" aspect="NONE" polarity="POS" pos="VERB" />
<MAKEINSTANCE eventID="e3" eiid="ei2409" tense="FUTURE" aspect="NONE" polarity="POS" pos="VERB" />
<MAKEINSTANCE eventID="e4" eiid="ei2410" tense="PRESENT" aspect="NONE" polarity="POS" pos="VERB" />
{Add more provided TimeML annotations here}
"""

# Replace placeholders in the template with actual text and annotations
time_ml_template = time_ml_template.replace("{Your text goes here}", text)
time_ml_template = time_ml_template.replace("{Your provided TimeML annotations go here}", provided_time_ml_annotations)

# Query for TimeML annotations
time_ml_annotation = query(time_ml_template).choices[0].message.content.strip()


with open('time_ml_annotation.xml', 'w') as file:
    file.write(time_ml_annotation)

output is:  ChatCompletion(id='chatcmpl-9EZLTZHHXlHfU7gM6kOkWHH4w7adl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Elon Reeve Musk was born on June 28, 1971. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world, with an estimated net worth of US$190 billion as of March 2024. \n\nIn 1995, he moved to California to attend Stanford University but dropped out after two days and co-founded online city guide software company Zip2. The startup was acquired by Compaq for $307 million in 1999. In 2002, eBay acquired PayPal for $1.5 billion, and that same year, Musk founded SpaceX with $100 million. In 2004, he became an early investor in Tesla Motors, Inc. In

In [6]:
template = (
    "Please provide the corresponding TimeML tag for each of the following entities: events, temporal expressions, and temporal relations. "
    "The output should be in the following format: \n"
    "ENTITY TYPE ID: Entity\n"
    "Where ENTITY TYPE can be one of the following: event (EVENT), time expression (TIMEX3), or temporal relation (TLINK).\n"
    "\n"
    "EVENT tag represents relevant events or states in the patient’s clinical timeline, such as “follow-up” or “admission”. It includes a “type” attribute to specify the event type and “modality” and “polarity” attributes to indicate the certainty and valuation of the event.\n"
    "TIMEX3 tag is used for temporal expressions, including dates, durations, times, and frequencies, such as “March 21, 2021” or “3 hours”. TIMEX3 tags’ additional attributes are “val”, that holds the normalized value of the time expression and “mod”, that holds the time modifier value.\n"
    "TLINK tag encodes three types of relations: overlap, before and after. TLINKs relate each event to the document’s admission or discharge date (“Section Time TLINK”). In addition, TLINKs connect EVENTS and TIMEX3s within the same sentence or across multiple sentences (“Non-Section Time TLINKs”).\n"
    "\n"
    "This is the text to be annotated: \n"
    f"{text}"
)

res = query(template).choices[0].message.content.strip()

output is:  ChatCompletion(id='chatcmpl-9EfKDYAm67GdL31KT4WCf0tEgfC5u', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='ENTITY TYPE ID: EVENT\nEVENT-1: Elon Reeve Musk is born on June 28, 1971\nEVENT-2: Elon Musk founds SpaceX in October 2002\nEVENT-3: Elon Musk becomes CEO of Tesla, Inc. in 2008\nEVENT-4: Elon Musk co-founds Neuralink in 2016\nEVENT-5: Elon Musk acquires Twitter in 2022\n\nENTITY TYPE ID: TIMEX3\nTIMEX3-1: June 28, 1971\nTIMEX3-2: March 2024\nTIMEX3-3: 1995\nTIMEX3-4: 1999\nTIMEX3-5: 2002\nTIMEX3-6: 2004\nTIMEX3-7: 2016\nTIMEX3-8: 2022\n\nENTITY TYPE ID: TLINK\nTLINK-1: EVENT-1 is BEFORE TIMEX3-1\nTLINK-2: EVENT-2 is AFTER TIMEX3-5\nTLINK-3: EVENT-3 is INCLUES TIMEX3-6\nTLINK-4: EVENT-4 is AFTER TIMEX3-7\nTLINK-5: EVENT-5 is AFTER TIMEX3-8', role='assistant', function_call=None, tool_calls=None))], created=1713282473, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=Com

In [5]:
template = (
    "Please provide the corresponding TimeML tag for each of the following entities: events, temporal expressions, and temporal relations. "
    "The output should be in the following format: \n"
    "ENTITY TYPE ID: Entity\n"
    "Where ENTITY TYPE can be one of the following: event (EVENT), time expression (TIMEX3), or temporal relation (TLINK).\n"
    "\n"
    "EVENT tag represents relevant events or states in the patient’s clinical timeline, such as “follow-up” or “admission”. It includes a “type” attribute to specify the event type and “modality” and “polarity” attributes to indicate the certainty and valuation of the event.\n"
    "TIMEX3 tag is used for temporal expressions, including dates, durations, times, and frequencies, such as “March 21, 2021” or “3 hours”. TIMEX3 tags’ additional attributes are “val”, that holds the normalized value of the time expression and “mod”, that holds the time modifier value.\n"
    "TLINK tag encodes three types of relations: overlap, before and after. TLINKs relate each event to the document’s admission or discharge date (“Section Time TLINK”). In addition, TLINKs connect EVENTS and TIMEX3s within the same sentence or across multiple sentences (“Non-Section Time TLINKs”).\n"
    "\n"
    "This is the text to be annotated: \n"
    f"{text}"
)

res = query(template, model="gpt-4-turbo").choices[0].message.content.strip()

output is:  ChatCompletion(id='chatcmpl-9EfaETa27sb3v2tVkBhTomEfQyNFF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='EVENT ID: founder\nEVENT ID: chairman\nEVENT ID: CEO\nEVENT ID: CTO\nEVENT ID: angel investor\nEVENT ID: product architect\nEVENT ID: former chairman\nEVENT ID: owner\nEVENT ID: executive chairman\nEVENT ID: co-founder\nEVENT ID: president\nEVENT ID: briefly attended\nEVENT ID: acquiring citizenship\nEVENT ID: matriculated\nEVENT ID: transferred\nEVENT ID: received\nEVENT ID: moved\nEVENT ID: attend\nEVENT ID: dropped out\nEVENT ID: co-founded\nEVENT ID: acquired\nEVENT ID: founded\nEVENT ID: became\nEVENT ID: helped create\nEVENT ID: proposed\nEVENT ID: acquired\nEVENT ID: merged\nEVENT ID: rebranded\nEVENT ID: founded\nEVENT ID: criticized\nEVENT ID: making\nEVENT ID: marked\nEVENT ID: changes\nEVENT ID: sued\nEVENT ID: stepped down\nEVENT ID: paid\nEVENT ID: was born\nEVENT ID: is\nEVENT ID: was elected\nEVENT ID